In [23]:
from BhuDM import *
from utils import *
import warnings
import yaml
try:
    from yaml import Cloader as Loader
except ImportError:

    from yaml import Loader

from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, QuantileTransformer
from tensorflow.keras import layers
from tensorflow.keras import models, callbacks, optimizers
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, QuantileTransformer
import matplotlib.pyplot as plt
import os
import pickle

In [24]:
config_file="phase2_paleotopography.yaml"
with open(config_file) as f:
    PARAMS = yaml.load(f, Loader=Loader)
print("––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––– ")
print(" Parameters set from %s" % config_file)
print("––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––– ")


––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––– 
 Parameters set from phase2_paleotopography.yaml
––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––– 


In [25]:
# Input Files 
MODEL_NAME=PARAMS['InputFiles']['plate_kinematics']['model_name']
MODEL_DIR = PARAMS['InputFiles']['plate_kinematics']['model_dir']  ## plate model
topology_filenames =[f"{MODEL_DIR}/{i}" for i in PARAMS['InputFiles']['plate_kinematics']['topology_files']]
rotation_filenames = [f"{MODEL_DIR}/{i}" for i in PARAMS['InputFiles']['plate_kinematics']['rotation_files']]
agegrid=PARAMS['InputFiles']['plate_kinematics']['agegrid']

ETOPO_FILE=PARAMS['InputFiles']['Raster']['ETOPO_FILE'] # ETOPO grid in meters (can be netCDf or GeoTiff)
ETOPO_Type=PARAMS['InputFiles']['Raster']['Raster_type']
coastlines = f"{MODEL_DIR }/{PARAMS['InputFiles']['plate_kinematics']['coastline_file']}"
static_polygon_file=f"{MODEL_DIR }/{PARAMS['InputFiles']['plate_kinematics']['static_polygon']}"
static_polygons = pygplates.FeatureCollection(static_polygon_file)
continents=f"{MODEL_DIR }/{PARAMS['InputFiles']['plate_kinematics']['continents']}"
print("––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––– ")
print("Reading input file..... \n")
print("––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––– ")
print(f"Plate Model: {MODEL_NAME} \n")
print(f"Model Directory: {MODEL_DIR} \n")
print(f"Coastlines: {coastlines} \n")
print(f"Continents: {continents} \n")
print(f"Static Polygons: {static_polygon_file} \n")
print(f"Model Agegrid: {agegrid} \n")
print(f"ETopo grid: {ETOPO_FILE}")
print("––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––– \n")

––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––– 
Reading input file..... 

––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––– 
Plate Model: phase2 

Model Directory: /Users/ssin4735/Documents/PROJECT/PhD Project/Codes and Data/Phase2 

Coastlines: /Users/ssin4735/Documents/PROJECT/PhD Project/Codes and Data/Phase2/StaticGeometries/Coastlines/Global_coastlines_low_res.shp 

Continents: /Users/ssin4735/Documents/PROJECT/PhD Project/Codes and Data/Phase2/StaticGeometries/ContinentalPolygons/Global_EarthByte_GPlates_PresentDay_ContinentsOnly.shp 

Static Polygons: /Users/ssin4735/Documents/PROJECT/PhD Project/Codes and Data/Phase2/StaticGeometries/StaticPolygons/Global_EarthByte_GPlates_PresentDay_StaticPlatePolygons.shp 

Model Agegrid: /Users/ssin4735/Documents/PROJECT/PhD Project/Codes and Data/Phase2/EarthByte_Plate_Motion_Model-Phase2-SeafloorAgeGrids-MantleFrame-NC 

ETopo grid: /Users/ssin4735/Documents/PROJECT/PhD Project/Codes and Data/

In [26]:
Paleomag_ID=PARAMS['Parameters']['paleomag_id']
Mantle_ID=PARAMS['Parameters']['mantle_optimised_id']

#The initial positions of crustal points are evenly distributed within the designated region. 
# At mesh refinement level zero, the points are approximately 20 degrees apart.
# Each increase in the density level results in a halving of the spacing between points.
MESH_REFINEMENT_LEVEL=PARAMS['Parameters']['mesh_refinement_level']  # higher refinement level will take longer time to run for optimisation 
WINDOW_SIZE=PARAMS['Parameters']['time_window_size']
Weighted=PARAMS['Parameters']['weighted_mean']


NETCDF_GRID_RESOLUTION=PARAMS['GridParameters']['grid_spacing']  # in degree
ZLIB=PARAMS['GridParameters']['compression']['zlib'] 
COMPLEVEL=PARAMS['GridParameters']['compression']['complevel'] 

FROM_TIME=int(PARAMS['TimeParameters']['time_max'])
TO_TIME=int(PARAMS['TimeParameters']['time_min'])
TIME_STEPS=int(PARAMS['TimeParameters']['time_step'])




parallel=PARAMS['Parameters']['number_of_cpus']### No of core to use or None for single core


print("––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––– ")
print("The following parameters are set-")
print("––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––– ")
print(f"Mantle Optmised Reference Frame ID: {Mantle_ID}")
print(f"Paleomagnetic Reference Frame ID: {Paleomag_ID} \n")

print(f"Moving Window Size: {WINDOW_SIZE}")
print(f"Weighted Mean: {Weighted}")

print(f"Mesh Refinement Level: {MESH_REFINEMENT_LEVEL}")
print(f"NetCDF GRID Resolution: {NETCDF_GRID_RESOLUTION}")
print(f"NetCDF Compression Level: {COMPLEVEL} \n")
print(f"Model Start Time: {FROM_TIME}")
print(f"Model End Time: {TO_TIME}")
print(f"Model Time Step: {TIME_STEPS}\n")


print(f"Number of CPU: {parallel}") # -1 means all the freely available CPU


print("––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––– ")

––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––– 
The following parameters are set-
––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––– 
Mantle Optmised Reference Frame ID: 702702
Paleomagnetic Reference Frame ID: 0 

Moving Window Size: 15.0
Weighted Mean: True
Mesh Refinement Level: 9
NetCDF GRID Resolution: 0.1
NetCDF Compression Level: 5 

Model Start Time: 525
Model End Time: 0
Model Time Step: 1

Number of CPU: -1
––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––– 


In [27]:
# Output Directory
OUTPUT_FOLDER=PARAMS['OutputFiles']['output_dir']

DEFAULT_OUTPUT_CSV=os.path.join(OUTPUT_FOLDER,'CSV')
DEFAULT_OUTPUT_NetCDF=os.path.join(OUTPUT_FOLDER,'NetCDF') # folder to store output NetCDF grid





print("––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––– ")
print(f"All the output will be saved in {OUTPUT_FOLDER}")
print("––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––– ")
create_directory_if_not_exists(OUTPUT_FOLDER)
create_directory_if_not_exists(DEFAULT_OUTPUT_CSV)
create_directory_if_not_exists(DEFAULT_OUTPUT_NetCDF)
print("––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––– ")


––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––– 
All the output will be saved in Paleotopography
––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––– 
––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––– 


## Define Plate Reconstruction

In [28]:
# rotation_model = pygplates.RotationModel(rotation_filenames)
# topology_features = pygplates.FeatureCollection()
# for topology_filename in topology_filenames:
#         topology_features.add( pygplates.FeatureCollection(topology_filename))


PK=PlateKinematicsParameters(topology_filenames, 
                             rotation_filenames,
                             static_polygons,
                             agegrid=agegrid,
                             coastlines=coastlines,
                             continents=continents,
                             anchor_plate_id=Mantle_ID)

time = 0 #Ma
gplot = gplately.PlotTopologies(PK.model, coastlines=coastlines, continents=continents, time=time)


RotationModel: No filename associated with <class 'pygplates.pygplates.RotationModel'> in __init__
 ensure pygplates is imported from gplately. Run,
 from gplately import pygplates


In [29]:
all_times=glob.glob(f"{DEFAULT_OUTPUT_CSV}/Prediction/*")
all_times=np.sort([int(time.split('_')[-1].split('.')[0].split('Ma')[0]) for time in all_times])

In [30]:
all_times

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 174, 175, 176, 177, 178, 179, 180, 181, 18

In [20]:
# all_sz_df=pd.read_parquet(f"{DEFAULT_OUTPUT_CSV}/ALL_Subduction.paraquet")

In [21]:
def poly_around_sub(i, subduction_df, n_steps=5,resolution=0.1):
    results = {
        'polygon':[],
        'dist':[],
        # 'trench_lats':[],
#         'trench_lons':[]
    }
   
    y1 = subduction_df.iloc[i]['Trench Latitude']
    y2 = subduction_df.iloc[i + 1]['Trench Latitude']
    x1 = subduction_df.iloc[i]['Trench Longitude']
    x2 = subduction_df.iloc[i + 1]['Trench Longitude']

    # dist = calc_dist(x1, y1, x2, y2)
    
    Point1=(x1,y1)
    Point2=(x2,y2)
    lines = LineString([Point1, Point2])
    dist=lines.length

    results['dist'].append(dist)

    # if  subduction_df.iloc[i]['Trench Plate ID']== subduction_df.iloc[i+1]['Trench Plate ID'] and dist <= 5.0:
    if  dist <= 5.0:
        try:

            ilon1=subduction_df.iloc[i]['Intersection Longitude'] 
            ilat1=subduction_df.iloc[i]['Intersection Latitude'] 
    
            ilon2=subduction_df.iloc[i+1]['Intersection Longitude'] 
            ilat2=subduction_df.iloc[i+1]['Intersection Latitude'] 
            
            coords = ((x1, y1), (x2, y2), (ilon2, ilat2), (ilon1, ilat1), (x1, y1))
            polygon = Polygon(coords)
            results['polygon'].append(polygon)
            # _, lats, lons = multipoints_from_polygon(polygon, resolution=0.09)
            # results['point_lats']=lats
            # results['point_lons']=lons

        
        except Exception as e:
            print(e)
            pass        
        
        return results
       
            

In [43]:
create_directory_if_not_exists(f"{DEFAULT_OUTPUT_NetCDF}/RF_Model")
create_directory_if_not_exists(f"{DEFAULT_OUTPUT_NetCDF}/DL_Model")
create_directory_if_not_exists(f"{DEFAULT_OUTPUT_NetCDF}/DLC_Model")
create_directory_if_not_exists(f"{DEFAULT_OUTPUT_NetCDF}/Ensemble_Model")
create_directory_if_not_exists(f"{DEFAULT_OUTPUT_NetCDF}/EBM_Model")
compression = {'zlib': ZLIB, 'complevel': COMPLEVEL}
for reconstruction_time in all_times:
    if reconstruction_time>=100:
        continue
    # try:
    print(f"Working on Time={reconstruction_time} Ma")
    Data=pd.read_parquet(f'{DEFAULT_OUTPUT_CSV}/Prediction/Predicted_{MODEL_NAME}_{reconstruction_time}Ma.parquet')
    column_for_netcdf1="ElevationRF"

    if compression:
        encoding = {column_for_netcdf1: compression}
    else:
        encoding = None
    
    da = df_to_NetCDF(x=Data['Longitude'], y=Data['Latitude'], z=Data[column_for_netcdf1], statistic='mean', grid_resolution=0.1, clip=(None, None))
    ds=da.to_dataset(name=column_for_netcdf1)
    ds.to_netcdf(f'{DEFAULT_OUTPUT_NetCDF}/RF_Model/RF_Model_{MODEL_NAME}_{reconstruction_time}.nc',encoding=encoding)
    
    column_for_netcdf2="ElevationDLC"
   
    if compression:
        encoding = {column_for_netcdf2: compression}
    else:
        encoding = None
    
    da = df_to_NetCDF(x=Data['Longitude'], y=Data['Latitude'], z=Data[column_for_netcdf2], statistic='mean', grid_resolution=0.1, clip=(None, None))
    ds=da.to_dataset(name=column_for_netcdf2)
    ds.to_netcdf(f'{DEFAULT_OUTPUT_NetCDF}/DLC_Model/DLC_Model_{MODEL_NAME}_{reconstruction_time}.nc',encoding=encoding)

    column_for_netcdf2="ElevationEBM"
   
    if compression:
        encoding = {column_for_netcdf2: compression}
    else:
        encoding = None
    
    da = df_to_NetCDF(x=Data['Longitude'], y=Data['Latitude'], z=Data[column_for_netcdf2], statistic='mean', grid_resolution=0.1, clip=(None, None))
    ds=da.to_dataset(name=column_for_netcdf2)
    ds.to_netcdf(f'{DEFAULT_OUTPUT_NetCDF}/EBM_Model/EBM_Model_{MODEL_NAME}_{reconstruction_time}.nc',encoding=encoding)

        
    
    
    column_for_netcdf2="ElevationDL"
   
    if compression:
        encoding = {column_for_netcdf2: compression}
    else:
        encoding = None
    
    da = df_to_NetCDF(x=Data['Longitude'], y=Data['Latitude'], z=Data[column_for_netcdf2], statistic='mean', grid_resolution=0.1, clip=(None, None))
    ds=da.to_dataset(name=column_for_netcdf2)
    ds.to_netcdf(f'{DEFAULT_OUTPUT_NetCDF}/DL_Model/DL_Model_{MODEL_NAME}_{reconstruction_time}.nc',encoding=encoding)

    column_for_netcdf3='Paleotopography'
   
    if compression:
        encoding = {column_for_netcdf3: compression}
    else:
        encoding = None
    
    da = df_to_NetCDF(x=Data['Longitude'], y=Data['Latitude'], z=Data[column_for_netcdf3], statistic='mean', grid_resolution=0.1, clip=(None, None))
    ds=da.to_dataset(name=column_for_netcdf3)
    ds.to_netcdf(f'{DEFAULT_OUTPUT_NetCDF}/Ensemble_Model/Ensemble_Model_{MODEL_NAME}_{reconstruction_time}.nc',encoding=encoding)

    print("Saved NetCDF!")
    # except Exception as e:
    #     print(e)

Created directory: Paleotopography/NetCDF/RF_Model
Created directory: Paleotopography/NetCDF/DL_Model
Created directory: Paleotopography/NetCDF/DLC_Model
Created directory: Paleotopography/NetCDF/Ensemble_Model
Created directory: Paleotopography/NetCDF/EBM_Model
Working on Time=0 Ma
Saved NetCDF!
Working on Time=1 Ma
Saved NetCDF!
Working on Time=2 Ma
Saved NetCDF!
Working on Time=3 Ma
Saved NetCDF!
Working on Time=4 Ma
Saved NetCDF!
Working on Time=5 Ma
Saved NetCDF!
Working on Time=6 Ma
Saved NetCDF!
Working on Time=7 Ma
Saved NetCDF!
Working on Time=8 Ma
Saved NetCDF!
Working on Time=9 Ma
Saved NetCDF!
Working on Time=10 Ma
Saved NetCDF!
Working on Time=11 Ma
Saved NetCDF!
Working on Time=12 Ma
Saved NetCDF!
Working on Time=13 Ma
Saved NetCDF!
Working on Time=14 Ma
Saved NetCDF!
Working on Time=15 Ma
Saved NetCDF!
Working on Time=16 Ma
Saved NetCDF!
Working on Time=17 Ma


KeyboardInterrupt: 

In [45]:
from shapely.geometry import LineString
import time
from gplately import Raster

create_directory_if_not_exists(f"{DEFAULT_OUTPUT_NetCDF}/Unmasked/RF_Model")
create_directory_if_not_exists(f"{DEFAULT_OUTPUT_NetCDF}/Unmasked/DL_Model")
create_directory_if_not_exists(f"{DEFAULT_OUTPUT_NetCDF}/Unmasked/DLC_Model")
create_directory_if_not_exists(f"{DEFAULT_OUTPUT_NetCDF}/Unmasked/Ensemble_Model")
create_directory_if_not_exists(f"{DEFAULT_OUTPUT_NetCDF}/Unmasked/EBM_Model")
n_steps=4

for reconstruction_time in all_times:
    # if reconstruction_time<8:
    #     continue
    print(f"Post Processing Time {reconstruction_time} Ma")
    # subduction_df = PK.get_subductiondf(reconstruction_time,tessellation_threshold_deg=0.7)
    # subduction_lines=[]
    # normal_angle=[]
    
    # for i in range(0, len(subduction_df)-1):
    #     Point1=(subduction_df.iloc[i]['Trench Longitude'],subduction_df.iloc[i]['Trench Latitude'])
    #     Point2=(subduction_df.iloc[i+1]['Trench Longitude'],subduction_df.iloc[i+1]['Trench Latitude'])
    #     lines = LineString([Point1, Point2])
    #     dist=lines.length
    #     # if  subduction_df.iloc[i]['Trench Plate ID']== subduction_df.iloc[i+1]['Trench Plate ID'] and dist<50.0:
    #     if  dist<=5.0:
    #         subduction_lines.append(lines)
    #     # else:
    #     #     subduction_lines.append(np.nan)
            
        
    # profiles=[]
    # trench_lats=[]
    # trench_lons=[]

    # for i in range(0, len(subduction_df)-1):
    #     dlon1 = n_steps * np.sin(np.radians(subduction_df.iloc[i]['Subduction Normal Angle']))
    #     dlat1 = n_steps * np.cos(np.radians(subduction_df.iloc[i]['Subduction Normal Angle']))
    #     x1=subduction_df.iloc[i]['Trench Longitude']
    #     y1=subduction_df.iloc[i]['Trench Latitude']
    #     ilon1 = x1 - dlon1
    #     ilat1 = y1- dlat1
        
        
    #     start_point = Point(x1- 0.01*dlon1, y1- 0.01*dlat1)
    #     end_point = Point(ilon1, ilat1)
    #     profile = LineString([start_point, end_point])
    #     profiles.append(profile)
    #     trench_lats.append(y1)
    #     trench_lons.append(x1) 
        
    # intersected_points=[]
    # intersected_profiles=[]
    # intersected_trench_lats=[]
    # intersected_trench_lons=[]
    # for i in range(len(profiles)):
    #     for subduction_line in subduction_lines:
            # try:
    #         intersection=profiles[i].intersection(subduction_line)
    #         if intersection.geom_type == 'Point':
    #             # print(profiles[i])
    #             intersected_profiles.append(profiles[i])
    #             intersected_trench_lats.append(trench_lats[i])
    #             intersected_trench_lons.append(trench_lons[i])
    #             intersected_points.append(intersection)
    #             # intersections2.append(intersection)
                
    # Intersection_df=gpd.GeoDataFrame()
    # Intersection_df['Trench Latitude I']=intersected_trench_lats
    # Intersection_df['Trench Longitude I']=intersected_trench_lons
    # Intersection_df['Intersection Point']=intersected_points
    # Intersection_df['Intersection Profile']=intersected_profiles
    # Intersection_df['Intersection Latitude']=Intersection_df['Intersection Point'].apply(lambda p: p.y)
    # Intersection_df['Intersection Longitude']=Intersection_df['Intersection Point'].apply(lambda p: p.x)
    # Intersection_df=Intersection_df.set_geometry(gpd.points_from_xy(Intersection_df['Trench Longitude I'],Intersection_df['Trench Latitude I']))
    # # Perform the spatial join
    # result = gpd.sjoin_nearest(subduction_df, Intersection_df, how='left', max_distance=1.0)
    # result=result.reset_index()
    # # Drop rows with the same index, keeping only the first occurrence
    # result = result.drop_duplicates(subset=['index'], keep='first')
    # result=result.set_index('index')
    # # Alternatively, drop duplicates based on the index column
    # # If you want to drop duplicates based on specific columns, replace 'index_left' with the appropriate column names
    
    # result ['Intersection Latitude'] = result ['Intersection Latitude'].fillna(result['Trench Latitude'] -n_steps*np.cos(np.radians(result['Subduction Normal Angle'])))
    # result ['Intersection Longitude'] = result ['Intersection Longitude'].fillna(result['Trench Longitude'] -n_steps*np.sin(np.radians(result['Subduction Normal Angle'])))
    
    
    # polygons=[]

    # results = Parallel(n_jobs=-1)(
    #     delayed(poly_around_sub)(i, result,resolution=0.1) for i in range(len(result) - 1)
    # )
    
    # for i, res in enumerate(results):
    #     try:
    #         polygons.extend(res['polygon'])

    #     except Exception as e:
    #         # print(e)
    #         pass
    
    Data=pd.read_parquet(f'{DEFAULT_OUTPUT_CSV}/Prediction/Predicted_{MODEL_NAME}_{reconstruction_time}Ma.parquet')
    # age_grid_file = find_filename_with_number(PK.agegrid,reconstruction_time)
    # print(age_grid_file)
    # oceanic_crust = rasterio.open(age_grid_file)
    # coordinates = [(x, y) for x, y in zip(Data['Longitude'].values, Data['Latitude'].values)]
    # oceanic_crust_point = list(oceanic_crust.sample(coordinates))
    # oceanic_crust_point=[oceanic_crust_point[i][0] for i in range(len(oceanic_crust_point))]
    # Data['Oceanic Crust']=oceanic_crust_point
    # Data = Data[pd.isna(Data['Oceanic Crust'])]
    # Data_gdf=gpd.GeoDataFrame(Data,geometry=gpd.points_from_xy(Data['Longitude'],Data['Latitude']))
    # Data_gdf=Data_gdf.set_crs("epsg:4326")
    # polygons_gdf=gpd.GeoDataFrame(geometry=polygons)
    # polygons_gdf=polygons_gdf.set_crs("epsg:4326")

    # # Ensure the coordinate reference systems match
    # if Data_gdf.crs != polygons_gdf.crs:
    #     Data_gdf = Data_gdf.to_crs(polygons_gdf.crs)
    # if 'index_right' in Data_gdf.columns:
    #     Data_gdf=Data_gdf.drop(columns=['index_right'])
    # # Perform spatial join to find points within polygons
    # points_within_polygons = gpd.sjoin(Data_gdf, polygons_gdf, predicate='within')
    
    # # Remove points that are within polygons from the original points GeoDataFrame
    # points_outside_polygons = Data_gdf.loc[~Data_gdf.index.isin(points_within_polygons.index)]
    # Data= points_outside_polygons
    
    
    
    column_for_netcdf="ElevationRF"
    
    da = df_to_NetCDF(x=Data['Longitude'], y=Data['Latitude'], z=Data[column_for_netcdf], statistic='mean', grid_resolution=0.1, clip=(None, None))
    
    raster=Raster(data=da, plate_reconstruction=PK.model, extent='global',  time=reconstruction_time)
    raster.rotate_reference_frames(grid_spacing_degrees=NETCDF_GRID_RESOLUTION,
                                   reconstruction_time=reconstruction_time, 
                                   from_rotation_features_or_model=PK.rotation_model, 
                                   to_rotation_features_or_model=PK.rotation_model, 
                                   from_rotation_reference_plate=Mantle_ID, 
                                   to_rotation_reference_plate=Paleomag_ID, 
                                   non_reference_plate=701, 
                                   output_name=f'{DEFAULT_OUTPUT_NetCDF}/Unmasked/RF_Model/RF_Model_{MODEL_NAME}_{reconstruction_time}.nc')



    column_for_netcdf="ElevationDL"
    
    da = df_to_NetCDF(x=Data['Longitude'], y=Data['Latitude'], z=Data[column_for_netcdf], statistic='mean', grid_resolution=0.1, clip=(None, None))
    
    raster=Raster(data=da, plate_reconstruction=PK.model, extent='global',  time=reconstruction_time)
    raster.rotate_reference_frames(grid_spacing_degrees=NETCDF_GRID_RESOLUTION,
                                   reconstruction_time=reconstruction_time, 
                                   from_rotation_features_or_model=PK.rotation_model, 
                                   to_rotation_features_or_model=PK.rotation_model, 
                                   from_rotation_reference_plate=Mantle_ID, 
                                   to_rotation_reference_plate=Paleomag_ID, 
                                   non_reference_plate=701, 
                                   output_name=f'{DEFAULT_OUTPUT_NetCDF}/Unmasked/DL_Model//DL_Model_{MODEL_NAME}_{reconstruction_time}.nc')


    column_for_netcdf="ElevationDLC"
    
    da = df_to_NetCDF(x=Data['Longitude'], y=Data['Latitude'], z=Data[column_for_netcdf], statistic='mean', grid_resolution=0.1, clip=(None, None))
    
    raster=Raster(data=da, plate_reconstruction=PK.model, extent='global',  time=reconstruction_time)
    raster.rotate_reference_frames(grid_spacing_degrees=NETCDF_GRID_RESOLUTION,
                                   reconstruction_time=reconstruction_time, 
                                   from_rotation_features_or_model=PK.rotation_model, 
                                   to_rotation_features_or_model=PK.rotation_model, 
                                   from_rotation_reference_plate=Mantle_ID, 
                                   to_rotation_reference_plate=Paleomag_ID, 
                                   non_reference_plate=701, 
                                   output_name=f'{DEFAULT_OUTPUT_NetCDF}/Unmasked/DLC_Model/DLC_Model_{MODEL_NAME}_{reconstruction_time}.nc')



    column_for_netcdf="Paleotopography"
    
    da = df_to_NetCDF(x=Data['Longitude'], y=Data['Latitude'], z=Data[column_for_netcdf], statistic='mean', grid_resolution=0.1, clip=(None, None))
    
    raster=Raster(data=da, plate_reconstruction=PK.model, extent='global',  time=reconstruction_time)
    raster.rotate_reference_frames(grid_spacing_degrees=NETCDF_GRID_RESOLUTION,
                                   reconstruction_time=reconstruction_time, 
                                   from_rotation_features_or_model=PK.rotation_model, 
                                   to_rotation_features_or_model=PK.rotation_model, 
                                   from_rotation_reference_plate=Mantle_ID, 
                                   to_rotation_reference_plate=Paleomag_ID, 
                                   non_reference_plate=701, 
                                   output_name=f'{DEFAULT_OUTPUT_NetCDF}/Unmasked/Ensemble_Model/Ensemble_Model_{MODEL_NAME}_{reconstruction_time}.nc')


    column_for_netcdf2="ElevationEBM"
   
    if compression:
        encoding = {column_for_netcdf2: compression}
    else:
        encoding = None
    
    da = df_to_NetCDF(x=Data['Longitude'], y=Data['Latitude'], z=Data[column_for_netcdf2], statistic='mean', grid_resolution=0.1, clip=(None, None))
    raster=Raster(data=da, plate_reconstruction=PK.model, extent='global',  time=reconstruction_time)
    raster.rotate_reference_frames(grid_spacing_degrees=NETCDF_GRID_RESOLUTION,
                                   reconstruction_time=reconstruction_time, 
                                   from_rotation_features_or_model=PK.rotation_model, 
                                   to_rotation_features_or_model=PK.rotation_model, 
                                   from_rotation_reference_plate=Mantle_ID, 
                                   to_rotation_reference_plate=Paleomag_ID, 
                                   non_reference_plate=701, 
                                   output_name=f'{DEFAULT_OUTPUT_NetCDF}/Unmasked/EBM_Model/EBM_Model_{MODEL_NAME}_{reconstruction_time}.nc')






Created directory: Paleotopography/NetCDF/Unmasked/RF_Model
Created directory: Paleotopography/NetCDF/Unmasked/DL_Model
Created directory: Paleotopography/NetCDF/Unmasked/DLC_Model
Created directory: Paleotopography/NetCDF/Unmasked/Ensemble_Model
Created directory: Paleotopography/NetCDF/Unmasked/EBM_Model
Post Processing Time 0 Ma
Post Processing Time 1 Ma
Post Processing Time 2 Ma
Post Processing Time 3 Ma
Post Processing Time 4 Ma
Post Processing Time 5 Ma
Post Processing Time 6 Ma
Post Processing Time 7 Ma
Post Processing Time 8 Ma
Post Processing Time 9 Ma
Post Processing Time 10 Ma
Post Processing Time 11 Ma
Post Processing Time 12 Ma
Post Processing Time 13 Ma
Post Processing Time 14 Ma
Post Processing Time 15 Ma
Post Processing Time 16 Ma
Post Processing Time 17 Ma
Post Processing Time 18 Ma
Post Processing Time 19 Ma
Post Processing Time 20 Ma
Post Processing Time 21 Ma
Post Processing Time 22 Ma
Post Processing Time 23 Ma
Post Processing Time 24 Ma
Post Processing Time 25 Ma


In [48]:
from shapely.geometry import LineString
import time
from gplately import Raster

create_directory_if_not_exists(f"{DEFAULT_OUTPUT_NetCDF}/Masked/RF_Model")
create_directory_if_not_exists(f"{DEFAULT_OUTPUT_NetCDF}/Masked/DL_Model")
create_directory_if_not_exists(f"{DEFAULT_OUTPUT_NetCDF}/Masked/DLC_Model")
create_directory_if_not_exists(f"{DEFAULT_OUTPUT_NetCDF}/Masked/Ensemble_Model")
create_directory_if_not_exists(f"{DEFAULT_OUTPUT_NetCDF}/Masked/EBM_Model")

n_steps=4

for reconstruction_time in all_times[:1]:
    print(f"Post Processing Time {reconstruction_time} Ma")
    # subduction_df = PK.get_subductiondf(reconstruction_time,tessellation_threshold_deg=0.7)
    # subduction_lines=[]
    # normal_angle=[]
    
    # for i in range(0, len(subduction_df)-1):
    #     Point1=(subduction_df.iloc[i]['Trench Longitude'],subduction_df.iloc[i]['Trench Latitude'])
    #     Point2=(subduction_df.iloc[i+1]['Trench Longitude'],subduction_df.iloc[i+1]['Trench Latitude'])
    #     lines = LineString([Point1, Point2])
    #     dist=lines.length
    #     # if  subduction_df.iloc[i]['Trench Plate ID']== subduction_df.iloc[i+1]['Trench Plate ID'] and dist<50.0:
    #     if  dist<=5.0:
    #         subduction_lines.append(lines)
    #     # else:
    #     #     subduction_lines.append(np.nan)
            
        
    # profiles=[]
    # trench_lats=[]
    # trench_lons=[]

    # for i in range(0, len(subduction_df)-1):
    #     dlon1 = n_steps * np.sin(np.radians(subduction_df.iloc[i]['Subduction Normal Angle']))
    #     dlat1 = n_steps * np.cos(np.radians(subduction_df.iloc[i]['Subduction Normal Angle']))
    #     x1=subduction_df.iloc[i]['Trench Longitude']
    #     y1=subduction_df.iloc[i]['Trench Latitude']
    #     ilon1 = x1 - dlon1
    #     ilat1 = y1- dlat1
        
        
    #     start_point = Point(x1- 0.01*dlon1, y1- 0.01*dlat1)
    #     end_point = Point(ilon1, ilat1)
    #     profile = LineString([start_point, end_point])
    #     profiles.append(profile)
    #     trench_lats.append(y1)
    #     trench_lons.append(x1) 
        
    # intersected_points=[]
    # intersected_profiles=[]
    # intersected_trench_lats=[]
    # intersected_trench_lons=[]
    # for i in range(len(profiles)):
    #     for subduction_line in subduction_lines:
    #         # try:
    #         intersection=profiles[i].intersection(subduction_line)
    #         if intersection.geom_type == 'Point':
    #             # print(profiles[i])
    #             intersected_profiles.append(profiles[i])
    #             intersected_trench_lats.append(trench_lats[i])
    #             intersected_trench_lons.append(trench_lons[i])
    #             intersected_points.append(intersection)
    #             # intersections2.append(intersection)
                
    # Intersection_df=gpd.GeoDataFrame()
    # Intersection_df['Trench Latitude I']=intersected_trench_lats
    # Intersection_df['Trench Longitude I']=intersected_trench_lons
    # Intersection_df['Intersection Point']=intersected_points
    # Intersection_df['Intersection Profile']=intersected_profiles
    # Intersection_df['Intersection Latitude']=Intersection_df['Intersection Point'].apply(lambda p: p.y)
    # Intersection_df['Intersection Longitude']=Intersection_df['Intersection Point'].apply(lambda p: p.x)
    # Intersection_df=Intersection_df.set_geometry(gpd.points_from_xy(Intersection_df['Trench Longitude I'],Intersection_df['Trench Latitude I']))
    # # Perform the spatial join
    # result = gpd.sjoin_nearest(subduction_df, Intersection_df, how='left', max_distance=1.0)
    # result=result.reset_index()
    # Drop rows with the same index, keeping only the first occurrence
    # result = result.drop_duplicates(subset=['index'], keep='first')
    # result=result.set_index('index')
    # # Alternatively, drop duplicates based on the index column
    # # If you want to drop duplicates based on specific columns, replace 'index_left' with the appropriate column names
    
    # result ['Intersection Latitude'] = result ['Intersection Latitude'].fillna(result['Trench Latitude'] -n_steps*np.cos(np.radians(result['Subduction Normal Angle'])))
    # result ['Intersection Longitude'] = result ['Intersection Longitude'].fillna(result['Trench Longitude'] -n_steps*np.sin(np.radians(result['Subduction Normal Angle'])))
    
    
    # polygons=[]

    # results = Parallel(n_jobs=-1)(
    #     delayed(poly_around_sub)(i, result,resolution=0.1) for i in range(len(result) - 1)
    # )
    
    # for i, res in enumerate(results):
    #     try:
    #         polygons.extend(res['polygon'])

    #     except Exception as e:
    #         # print(e)
    #         pass
    
    Data=pd.read_parquet(f'{DEFAULT_OUTPUT_CSV}/Prediction/Predicted_{MODEL_NAME}_{reconstruction_time}Ma.parquet')
    age_grid_file = find_filename_with_number(PK.agegrid,reconstruction_time)
    print(age_grid_file)
    oceanic_crust = rasterio.open(age_grid_file)
    coordinates = [(x, y) for x, y in zip(Data['Longitude'].values, Data['Latitude'].values)]
    oceanic_crust_point = list(oceanic_crust.sample(coordinates))
    oceanic_crust_point=[oceanic_crust_point[i][0] for i in range(len(oceanic_crust_point))]
    Data['Oceanic Crust']=oceanic_crust_point
    Data = Data[pd.isna(Data['Oceanic Crust'])]
    # Data_gdf=gpd.GeoDataFrame(Data,geometry=gpd.points_from_xy(Data['Longitude'],Data['Latitude']))
    # Data_gdf=Data_gdf.set_crs("epsg:4326")
    # polygons_gdf=gpd.GeoDataFrame(geometry=polygons)
    # polygons_gdf=polygons_gdf.set_crs("epsg:4326")

    # # Ensure the coordinate reference systems match
    # if Data_gdf.crs != polygons_gdf.crs:
    #     Data_gdf = Data_gdf.to_crs(polygons_gdf.crs)
    # if 'index_right' in Data_gdf.columns:
    #     Data_gdf=Data_gdf.drop(columns=['index_right'])
    # # Perform spatial join to find points within polygons
    # points_within_polygons = gpd.sjoin(Data_gdf, polygons_gdf, predicate='within')
    
    # # Remove points that are within polygons from the original points GeoDataFrame
    # points_outside_polygons = Data_gdf.loc[~Data_gdf.index.isin(points_within_polygons.index)]
    # Data= points_outside_polygons
    
    
    
    column_for_netcdf="ElevationRF"
    
    da = df_to_NetCDF(x=Data['Longitude'], y=Data['Latitude'], z=Data[column_for_netcdf], statistic='mean', grid_resolution=0.1, clip=(None, None))
    
    raster=Raster(data=da, plate_reconstruction=PK.model, extent='global',  time=reconstruction_time)
    raster.rotate_reference_frames(grid_spacing_degrees=NETCDF_GRID_RESOLUTION,
                                   reconstruction_time=reconstruction_time, 
                                   from_rotation_features_or_model=PK.rotation_model, 
                                   to_rotation_features_or_model=PK.rotation_model, 
                                   from_rotation_reference_plate=Mantle_ID, 
                                   to_rotation_reference_plate=Paleomag_ID, 
                                   non_reference_plate=701, 
                                   output_name=f'{DEFAULT_OUTPUT_NetCDF}/Masked/RF_Model/RF_Model_{MODEL_NAME}_{reconstruction_time}.nc')



    column_for_netcdf="ElevationDL"
    
    da = df_to_NetCDF(x=Data['Longitude'], y=Data['Latitude'], z=Data[column_for_netcdf], statistic='mean', grid_resolution=0.1, clip=(None, None))
    
    raster=Raster(data=da, plate_reconstruction=PK.model, extent='global',  time=reconstruction_time)
    raster.rotate_reference_frames(grid_spacing_degrees=NETCDF_GRID_RESOLUTION,
                                   reconstruction_time=reconstruction_time, 
                                   from_rotation_features_or_model=PK.rotation_model, 
                                   to_rotation_features_or_model=PK.rotation_model, 
                                   from_rotation_reference_plate=Mantle_ID, 
                                   to_rotation_reference_plate=Paleomag_ID, 
                                   non_reference_plate=701, 
                                   output_name=f'{DEFAULT_OUTPUT_NetCDF}/Masked/DL_Model//DL_Model_{MODEL_NAME}_{reconstruction_time}.nc')


    column_for_netcdf="ElevationDLC"
    
    da = df_to_NetCDF(x=Data['Longitude'], y=Data['Latitude'], z=Data[column_for_netcdf], statistic='mean', grid_resolution=0.1, clip=(None, None))
    
    raster=Raster(data=da, plate_reconstruction=PK.model, extent='global',  time=reconstruction_time)
    raster.rotate_reference_frames(grid_spacing_degrees=NETCDF_GRID_RESOLUTION,
                                   reconstruction_time=reconstruction_time, 
                                   from_rotation_features_or_model=PK.rotation_model, 
                                   to_rotation_features_or_model=PK.rotation_model, 
                                   from_rotation_reference_plate=Mantle_ID, 
                                   to_rotation_reference_plate=Paleomag_ID, 
                                   non_reference_plate=701, 
                                   output_name=f'{DEFAULT_OUTPUT_NetCDF}/Masked/DLC_Model/DLC_Model_{MODEL_NAME}_{reconstruction_time}.nc')



    column_for_netcdf="Paleotopography"
    
    da = df_to_NetCDF(x=Data['Longitude'], y=Data['Latitude'], z=Data[column_for_netcdf], statistic='mean', grid_resolution=0.1, clip=(None, None))
    
    raster=Raster(data=da, plate_reconstruction=PK.model, extent='global',  time=reconstruction_time)
    raster.rotate_reference_frames(grid_spacing_degrees=NETCDF_GRID_RESOLUTION,
                                   reconstruction_time=reconstruction_time, 
                                   from_rotation_features_or_model=PK.rotation_model, 
                                   to_rotation_features_or_model=PK.rotation_model, 
                                   from_rotation_reference_plate=Mantle_ID, 
                                   to_rotation_reference_plate=Paleomag_ID, 
                                   non_reference_plate=701, 
                                   output_name=f'{DEFAULT_OUTPUT_NetCDF}/Masked/Ensemble_Model/Ensemble_Model_{MODEL_NAME}_{reconstruction_time}.nc')


    column_for_netcdf2="ElevationEBM"
   
    if compression:
        encoding = {column_for_netcdf2: compression}
    else:
        encoding = None
    
    da = df_to_NetCDF(x=Data['Longitude'], y=Data['Latitude'], z=Data[column_for_netcdf2], statistic='mean', grid_resolution=0.1, clip=(None, None))
    raster=Raster(data=da, plate_reconstruction=PK.model, extent='global',  time=reconstruction_time)
    raster.rotate_reference_frames(grid_spacing_degrees=NETCDF_GRID_RESOLUTION,
                                   reconstruction_time=reconstruction_time, 
                                   from_rotation_features_or_model=PK.rotation_model, 
                                   to_rotation_features_or_model=PK.rotation_model, 
                                   from_rotation_reference_plate=Mantle_ID, 
                                   to_rotation_reference_plate=Paleomag_ID, 
                                   non_reference_plate=701, 
                                   output_name=f'{DEFAULT_OUTPUT_NetCDF}/Masked/EBM_Model/EBM_Model_{MODEL_NAME}_{reconstruction_time}.nc')









Post Processing Time 0 Ma
/Users/ssin4735/Documents/PROJECT/PhD Project/Codes and Data/Phase2/EarthByte_Plate_Motion_Model-Phase2-SeafloorAgeGrids-MantleFrame-NC/EarthByte_Plate_Motion_Model-Phase2-MantleReferenceFrame-0.nc


In [ ]:
import os
import numpy as np
import xarray as xr

def create_directory_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def interpolate_and_save_as_nc(start_time, end_time, required_time_step=1):
    initial_time_step = int(end_time - start_time)
    
    try:
        # Open the NetCDF files with xarray
        src_start = xr.open_dataset(f"/Users/ssin4735/Documents/PROJECT/PhD Project/Codes and Data/DeepTimeTopo/Workflow/Paleotopography/NetCDF/RF/RF_Model_phase2_{start_time}.nc")
        src_end = xr.open_dataset(f"/Users/ssin4735/Documents/PROJECT/PhD Project/Codes and Data/DeepTimeTopo/Workflow/Paleotopography/NetCDF/RF/RF_Model_phase2_{end_time}.nc")

        # Get the data variable name (replace 'ElevationRF (Ca)' with the actual data variable name if different)
        data_variable_name = 'Paleotopography'  # Ensure this matches the variable name in your files
        start_data = src_start[data_variable_name]
        end_data = src_end[data_variable_name]

        # Extract lat/lon variables
        latitudes = src_start['Latitude']
        longitudes = src_start['Longitude']

        # Define the interpolation time steps
        time_steps = range(1, initial_time_step, required_time_step)

        for time in time_steps:
            # Calculate the interpolated data using your formula
            time_fraction = time / float(initial_time_step)
            interpolated_data = start_data + (end_data - start_data) * time_fraction

            # Prepare NetCDF parameters
            output_nc = f"/Users/ssin4735/Documents/PROJECT/PhD Project/Codes and Data/DeepTimeTopo/Workflow/Paleotopography/NetCDF/RF/RF_Model_phase2_{start_time+time}.nc"
            create_directory_if_not_exists(os.path.dirname(output_nc))
            
            # Create a new xarray Dataset for the interpolated data
            ds_interpolated = xr.Dataset(
                {
                    data_variable_name: (['Latitude', 'Longitude'], interpolated_data.values),
                },
                coords={
                    'Latitude': latitudes,
                    'Longitude': longitudes,
                }
            )
            
            # Copy attributes from source files
            ds_interpolated.attrs.update(src_start.attrs)
            ds_interpolated[data_variable_name].attrs.update(src_start[data_variable_name].attrs)
            
            # Save to NetCDF with compression
            ds_interpolated.to_netcdf(output_nc, mode='w', engine='netcdf4', format='NETCDF4', encoding={data_variable_name: {'zlib': True, 'complevel': 5}})
            
        print(f"Interpolation completed from {start_time} Ma to {end_time} Ma.")

    except Exception as e:
        print(f"No raster file for interpolating from {start_time} Ma to {end_time} Ma: {e}")

# Example usage:
# i = 10
# all_times = [13, 14, 15]  # Example times, update with your actual time steps
# interpolate_and_save_as_nc(start_time=all_times[i], end_time=all_times[i+1], required_time_step=1)
all_times=glob.glob("/Users/ssin4735/Documents/PROJECT/PhD Project/Codes and Data/DeepTimeTopo/Workflow/Paleotopography//NetCDF/RF/RF_Model_phase2_*.nc")

# all_times=glob.glob(f"{DEFAULT_OUTPUT_CSV}/Processed_WMA3/*")
all_times=np.sort([int(time.split('_')[-1].split('.')[0]) for time in all_times])
for i in range(len(all_times)-1):
    interpolate_and_save_as_nc(start_time=all_times[i], end_time=all_times[i+1], required_time_step=1)